<a href="https://colab.research.google.com/github/skolix15/Machine_Learning_2025/blob/main/Exercise_six_(6).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import sys
!{sys.executable} -m pip install -U ydata-profiling[notebook]
!pip install jupyter-contrib-nbextensions
!jupyter nbextension enable --py widgetsnbextension

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Question 1

In [4]:
# Load Data
df = pd.read_csv("bankloan.csv")
# Create profile report object
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
# # Appear to notebook
# profile.to_notebook_iframe()
# Save to HTML file
# profile.to_file("notebook_report.html")

# Question 2

## Subquestion 1

Mean: 15116.256  

Max: 35000

Min: 1000

## Subquestion 2

Μπορούμε να αφαιρέσουμε τις παρακάτω μεταβλητές:

(a) Μεταβλητές που λειτουργούν ως αναγνωριστικά =>
    1. id (μοναδικές τιμές),
    2. mebmer_id (μοναδικές τιμές),
    3. row_id (μοναδικές τιμές),
    4. unnamed (είναι μη υποστηριζόμενου τύπου και έχει 100% ελλειπούσες τιμές)

(b) Μεταβλητές με μεγάλο ποσοστό ελλειπούσων τιμώ =>
    5. annual_inc_joint (99.9% ελλειπούσες τιμές)
    6. dti_joint (99.9% ελλειπούσες τιμές)
    7. 36months (86.1% ελλειπούσες τιμές, είναι υψηλά συσχετισμένη με τις μεταβλητές 60months & term)
    8. 60months (86.1% ελλειπούσες τιμές, είναι υψηλά συσχετισμένη με τις μεταβλητές 36months & term)
    9. next_pymnt_d (75.5% ελλειπούσες τιμές και μη ισρροπημένο -> 98.8%)
    10. mths_since_last_major_derog (71.8% ελλειπούσες τιμές)

(c) Μεταβλητές με υψηλή συσχέτιση =>
    11. title (υψηλά συσχετισμένη με τη μεταβλητή purpose, μπορεί να αφαιρεθεί διατηρώντας την purpose)
    12. grade & sub_grade (υψηλά συσχετισμένες μεταξύ τους. Ενδεχομένως, να αρκούσε η μία από τις δύο)
    13. funded_amnt & loan_amnt (υψηλά συσχετισμένες μεταξύ τους. Ενδεχομένως, να αρκούσε η μία από τις δύο)

In [5]:
# Set variable to drop
variables_to_drop = [

  # Identifiers/Non-predictive features
  'id', 'member_id', 'Row ID', 'Unnamed: 50',

  # Variables with high percentage of missing values (>70%)
  'annual_inc_joint', 'dti_joint', '36months', '60months',
  'next_pymnt_d', 'mths_since_last_major_derog', # ~71.8% missing

  # Redundant/Highly correlated variables
  'title', 'grade', 'funded_amnt'

]

# Drop the specified columns from the DataFrame
df_cleaned = df.drop(columns=variables_to_drop, axis=1)

# Print information
print(f"\nVariables dropped: {variables_to_drop}")
print(f"Cleaned DataFrame dimensions: {df_cleaned.shape}")


Variables dropped: ['id', 'member_id', 'Row ID', 'Unnamed: 50', 'annual_inc_joint', 'dti_joint', '36months', '60months', 'next_pymnt_d', 'mths_since_last_major_derog', 'title', 'grade', 'funded_amnt']
Cleaned DataFrame dimensions: (212999, 40)


## Subquestion 3

In [6]:
# Fill undefined/null values

# Get numerical columns
numerical_columns = df_cleaned.select_dtypes(include=['number']).columns

# Get categorical columns
categorical_columns = df_cleaned.select_dtypes(include=['object']).columns

# Fill empty numerical columns with mean value
for column in numerical_columns:
  if df_cleaned[column].isnull().any():
          mean_value = df_cleaned[column].mean()
          df_cleaned[column].fillna(mean_value)

# Fill empty categorical columns with most used value (mode)
for column in categorical_columns:
    if df_cleaned[column].isnull().any():
        # Using .iloc[0] in order to get the first value, in case of multiple modes
        mode_value = df_cleaned[column].mode().iloc[0]
        df_cleaned[column].fillna(mode_value)

# Store updated/cleaned dataframe in new csv file
df_cleaned.to_csv("bankloan_cleaned.csv", index=False)
print("Updated/cleaned Dataframe was stored in 'bankloan_cleaned.csv' file!")

Updated/cleaned Dataframe was stored in 'bankloan_cleaned.csv' file!


## Subquestion 4

In [7]:
# Set target values/subgrades
target_sub_grades = ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2']

# Create target column with values 0 and 1
# Target: 1 => if sub_grade is in target_sub_grades
# Target: 0 =>  otherwise
df_cleaned['target_loan_risk'] = df_cleaned['sub_grade'].apply(lambda x: 1 if x in target_sub_grades else 0)

# Calculate target counts & perchentages
target_counts = df_cleaned['target_loan_risk'].value_counts()
target_percentages = (df_cleaned['target_loan_risk'].value_counts(normalize=True) * 100).round(2).astype(str) + ' %'

# Print results
print(f"Target Counts:\n{target_counts.to_string(header=False)}\n")
print(f"Target Percentages:\n{target_percentages.to_string(header=False)}\n")

Target Counts:
0    151709
1     61290

Target Percentages:
0    71.23 %
1    28.77 %



Με βάση τα παραπάνω στατιστικά, παρατηρούμε ότι η μεταβλητή target παρουσιάζει σημαντική ανισορροπία. Η κλάση 0 (Υψηλότερος Κίνδυνος/Non-Target) κυριαρχεί με 73.06%, ενώ η κλάση 1 (Χαμηλός Κίνδυνος/Target) αποτελεί τη μειοψηφία (26.94%).

## Subquestion 5

In [8]:
# Create ranges (loan_amoun_ranges) of 2000 from the minimum to the maximum loan amount.
loan_amount_ranges = range(
    int(df_cleaned['loan_amnt'].min()),
    int(df_cleaned['loan_amnt'].max()) + 2000,
    2000
)


# Categorize each loan into the corresponding load amount ranges
# pd.cut assigns every loan_amnt value to one of the defined intervals.
df_cleaned['loan_range'] = pd.cut(df_cleaned['loan_amnt'], loan_amount_ranges)

# Calculate approval rate per loan amount range
approval_rate = df_cleaned.groupby('loan_range')['target_loan_risk'].mean()

# Filter only the intervals where the approval probability
# is at least 0.15 (15%).
desired_amount_ranges = approval_rate[approval_rate >= 0.15]

# Print desired amount ranges
print(f"Desired amount ranges:\n\n{desired_amount_ranges}\n")


Desired amount ranges:

loan_range
(1000, 3000]      0.182740
(3000, 5000]      0.283272
(5000, 7000]      0.347792
(7000, 9000]      0.349951
(9000, 11000]     0.334011
(11000, 13000]    0.294425
(13000, 15000]    0.296526
(15000, 17000]    0.270516
(17000, 19000]    0.262080
(19000, 21000]    0.287494
(21000, 23000]    0.217605
(23000, 25000]    0.322029
(25000, 27000]    0.241466
(27000, 29000]    0.395797
(29000, 31000]    0.173149
Name: target_loan_risk, dtype: float64



/tmp/ipython-input-3636721772.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  approval_rate = df_cleaned.groupby('loan_range')['target_loan_risk'].mean()


# Question 3

## Normalization Method

Για την κανονικοποίηση των εισόδων στο πρόβλημα πρόβλεψης δανειοδότησης, επέλεξα τη μέθοδο StandardScaler, η οποία μετασχηματίζει κάθε αριθμητική μεταβλητή ώστε να έχει μέση τιμή 0 και τυπική απόκλιση 1. Η επιλογή αυτή έγινε διότι τα χαρακτηριστικά διαφέρουν σημαντικά σε κλίμακα και μονάδες μέτρησης, όπως για παράδειγμα το ποσό δανείου και το ετήσιο εισόδημα. Με αυτόν τον τρόπο τα μοντέλα που βασίζονται σε απόσταση ή gradient, όπως η Logistic Regression, το SVM ή το KNN, μπορούν να συγκρίνουν τις μεταβλητές με ίση βαρύτητα, ενώ ταυτόχρονα διατηρείται η σχετική κατανομή των τιμών και δεν περιορίζονται αυθαίρετα οι ακραίες τιμές, όπως θα συνέβαινε με το MinMaxScaler.

## Source Code

In [9]:
# ---- 0) Precondition: df_cleaned exists and 'sub_grade' & 'target_loan_risk' present ----
# Drop leakage
df_model = df_cleaned.drop(columns=['sub_grade'], axis=1)

# Features / target
X = df_model.drop(columns=['target_loan_risk'])
y = df_model['target_loan_risk']

# Identify feature types
num_features = X.select_dtypes(include=['number']).columns.tolist()
cat_features = X.select_dtypes(include=['object','category']).columns.tolist()

# --- 1) Build preprocessing pipelines ---
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),   # fill numeric NaNs
    ('scaler', StandardScaler())                     # normalize -> mean=0, std=1
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # fill categorical NaNs
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True, drop='first'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, num_features),
        ('cat', categorical_pipeline, cat_features)
    ],
    # preserve sparse output if many features are sparse; leave default sparse_threshold
)

# --- 2) Full pipeline with RandomForest ---
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
pipeline = Pipeline([
    ('preproc', preprocessor),
    ('clf', rf)
])

# --- 3) Train/test split (stratify) ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# --- 4) GridSearch (light) on n_estimators ---
param_grid = {'clf__n_estimators': [50, 100]}  # keep small for Colab
grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='f1', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)

# --- 5) Evaluate on test set ---
y_pred = grid.predict(X_test)

print("Best params:", grid.best_params_)
print("Accuracy : ", round(accuracy_score(y_test, y_pred), 3))
print("Precision: ", round(precision_score(y_test, y_pred), 3))
print("Recall   : ", round(recall_score(y_test, y_pred), 3))
print("F1-score : ", round(f1_score(y_test, y_pred), 3))




Fitting 3 folds for each of 2 candidates, totalling 6 fits


/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [1, 8] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Best params: {'clf__n_estimators': 100}
Accuracy :  0.995
Precision:  1.0
Recall   :  0.984
F1-score :  0.992


## Classifiter Selection

Επέλεξα RandomForest γιατί το dataset είναι μεγάλο και περιέχει τόσο αριθμητικά όσο και κατηγορικά χαρακτηριστικά. Το RandomForest ανιχνεύει μη γραμμικές σχέσεις, είναι ανθεκτικό σε outliers, και συνήθως έχει καλύτερη απόδοση σε πολύπλοκα datasets. Η παράμετρος n_estimators ρυθμίστηκε μέσω cross-validation, καθώς είναι η πιο ευαίσθητη παράμετρος για το μοντέλο.

## Most crucial metric

Η πιο σημαντική μετρική για αυτή την εφαρμογή είναι το F1-score, καθώς εξισορροπεί Precision και Recall. Εναλλακτικά, ανάλογα με την προτεραιότητα της τράπεζας, μπορούμε να δώσουμε μεγαλύτερη έμφαση στο Precision για να ελαχιστοποιήσουμε τον κίνδυνο κακών δανειοληπτών.

# Question 4

# Question 5